In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from operator import itemgetter

In [8]:
def length_function(text):                      # 텍스트의 길이를 반환하는 함수
    return len(text)

In [7]:
def _multiple_length_function(text1, text2):    # 두 텍스트의 길이를 곱하는 함수
    return len(text1) * len(text2)

In [12]:
def multiple_length_function(_dict):            # 2개 인자를 받는 함수로 연결하는 wrapper 함수
    # _dict["text1"] 값 _dict["text2"] 값을 인자로 넘긴다.
    return _multiple_length_function(_dict["text1"], _dict["text2"])    

In [13]:
prompt = ChatPromptTemplate.from_template("what is {a} + {b}?")

In [14]:
llm = ChatOpenAI(model_name="gpt-4o-mini")

In [15]:
output_parser = StrOutputParser()

In [17]:
chain1 = prompt | llm | output_parser   # 프롬프트와 모델을 연결하여 체인 생성

In [18]:
chain = (
    {
        "a": itemgetter("input_1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("input_1"), "text2": itemgetter("input_2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | llm
    | output_parser
)

In [19]:
# 주어진 인자들로 체인을 실행합니다.
chain.invoke({"input_1": "bar", "input_2": "gah"})

'3 + 9 equals 12.'